# PW2 - VAE for MNIST clustering and generation

In this practical, we will explore Variational Auto-Encoders (VAE) with the MNIST digit recognition dataset.
The purpose of a VAE is to generate new samples "looking like" the training samples.
For this purpose, we will construct a two-block model: encoder and decoder, one takes a sample and maps it to a latent space, the second takes a latent point and maps it to the sample space. If the latent distribution is roughly gaussian, this gives a way to get new samples: take a latent point at random,
and map it to the sample space with the decoder.

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
%matplotlib inline

from sklearn.metrics.cluster import normalized_mutual_info_score

In [ ]:
N_CLASSES = 10
IMAGE_SIZE = 28 * 28

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
def show(img):
    plt.imshow(np.transpose(img.numpy(), (1,2,0)), interpolation='nearest')

def plot_reconstruction(model, n=24):
    x,_ = next(iter(data_loader))
    x = x[:n,:,:,:].to(device)
    try:
        out, _, _, log_p = model(x.view(-1, IMAGE_SIZE))
    except:
        try:
            out, _, _ = model(x.view(-1, IMAGE_SIZE))
        except:
            out, _, _, _, _ = model(x.view(-1, IMAGE_SIZE))
    x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
    out_grid = torchvision.utils.make_grid(x_concat).cpu().data
    show(out_grid)

@torch.no_grad()
def plot_generation(model, n=24):
    z = torch.randn(n, z_dim).to(device)
    out = model.decode(z).view(-1, 1, 28, 28)
    out_grid = torchvision.utils.make_grid(out).cpu()
    show(out_grid)

@torch.no_grad()
def plot_conditional_generation(model, n=8, fix_number=None):
    matrix = np.zeros((n, N_CLASSES))
    matrix[:,0] = 1

    if fix_number is None:
        final = matrix[:]
        for i in range(1, N_CLASSES):
            final = np.vstack((final,np.roll(matrix,i)))
        z = torch.randn(n, z_dim)
        z = z.repeat(N_CLASSES,1).to(device)
        y_onehot = torch.tensor(final).type(torch.FloatTensor).to(device)
        out = model.decode(z,y_onehot).view(-1, 1, 28, 28)
    else:
        z = torch.randn(n, z_dim).to(device)
        y_onehot = torch.tensor(np.roll(matrix, fix_number)).type(torch.FloatTensor).to(device)
        out = model.decode(z,y_onehot).view(-1, 1, 28, 28)

    out_grid = torchvision.utils.make_grid(out).cpu()
    show(out_grid)

@torch.no_grad()
def live_plot(data_dict, x_key=None, figsize=(7,5), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    for label, data in data_dict.items():
        if label == x_key or len(data) == 0:
            continue
        x = data_dict[x_key] if x_key is not None else np.arange(len(data))
        plt.plot(x, data, label=label, linewidth=1)
    plt.title(title)
    plt.grid(alpha=.5, which='both')
    plt.xlabel('epoch' if x_key is None else x_key)
    plt.legend(loc='center left') # the plot evolves to the right
    plt.show();

In [ ]:
data_dir = './data'
dataset = torchvision.datasets.MNIST(
    root=data_dir,
    train=True,
    transform=transforms.ToTensor(),
    download=True)

data_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=128,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(data_dir, train=False, download=True, transform=transforms.ToTensor()),
    batch_size=10, shuffle=False)

# Part A - Variational Autoencoders

## A.1 - Autoencoding theory

Consider a latent variable model with a data variable $x\in \mathcal{X}$ and a latent variable $z\in \mathcal{Z}$, $p(z,x) = p(z)p_\theta(x|z)$. Given the data $x_1,\dots, x_n$, we want to train the model by maximizing the marginal log-likelihood:
\begin{eqnarray*}
\mathcal{L} = \mathbf{E}_{p_d(x)}\left[\log p_\theta(x)\right]=\mathbf{E}_{p_d(x)}\left[\log \int_{\mathcal{Z}}p_{\theta}(x|z)p(z)dz\right],
  \end{eqnarray*}
  where $p_d$ denotes the empirical distribution of $X$: $p_d(x) =\frac{1}{n}\sum_{i=1}^n \delta_{x_i}(x)$.

 To avoid the (often) difficult computation of the integral above, the idea behind variational methods is to instead maximize a lower bound to the log-likelihood:
  \begin{eqnarray*}
\mathcal{L} \geq L(p_\theta(x|z),q(z|x)) =\mathbf{E}_{p_d(x)}\left[\mathbf{E}_{q(z|x)}\left[\log p_\theta(x|z)\right]-\mathrm{KL}\left( q(z|x)||p(z)\right)\right].
  \end{eqnarray*}
  Any choice of $q(z|x)$ gives a valid lower bound. Variational autoencoders replace the variational posterior $q(z|x)$ by an inference network $q_{\phi}(z|x)$ that is trained together with $p_{\theta}(x|z)$ to jointly maximize $L(p_\theta,q_\phi)$.
  
The variational posterior $q_{\phi}(z|x)$ is also called the **encoder** and the generative model $p_{\theta}(x|z)$, the **decoder** or generator.

The first term $\mathbf{E}_{q(z|x)}\left[\log p_\theta(x|z)\right]$ is the negative reconstruction error. Indeed under a gaussian assumption i.e. $p_{\theta}(x|z) = \mathcal{N}(\mu_{\theta}(z), I)$ the term $\log p_\theta(x|z)$ reduces to $\propto \|x-\mu_\theta(z)\|^2$, which is often used in practice. The term $\mathrm{KL}\left( q(z|x)||p(z)\right)$ can be seen as a regularization term, where the variational posterior $q_\phi(z|x)$ should be matched to the prior $p(z)= \mathcal{N}(0, I)$.

Variational Autoencoders were introduced by [Kingma and Welling (2013)](https://arxiv.org/abs/1312.6114), see also [(Doersch, 2016)](https://arxiv.org/abs/1606.05908) for a tutorial.

## A.2 - A simple autoencoder for MNIST

Implement the following autoencoder.


![A variational autoencoder.](https://github.com/dataflowr/notebooks/blob/master/HW3/vae.png?raw=true)

Add a ReLU after FC1 and FC4


$\mu$ and $\log \sigma^2$ are used for reparametrization with
$z= \mu + \sigma \odot \epsilon$, where $\epsilon \sim \mathcal{N}(0,1)$

> Left is without the “reparameterization trick”, and right is with it. Red shows sampling operations that are non-differentiable. Blue shows loss layers. The feedforward behavior of these networks is identical, but backpropagation can be applied only to the right network.



In [ ]:
# VAE model
class VAE(nn.Module):
    def __init__(self, h_dim, z_dim):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(IMAGE_SIZE, h_dim)
        self.fc_mu = nn.Linear(h_dim, z_dim)
        self.fc_log_var = nn.Linear(h_dim, z_dim)

        self.fc3 = nn.Linear(z_dim, h_dim)
        self.fc4 = nn.Linear(h_dim, IMAGE_SIZE)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        mu = self.fc_mu(h)
        log_var = self.fc_log_var(h)
        return mu, log_var

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h))

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_rec = self.decode(z)
        return x_rec, mu, log_var, z

# Hyper-parameters
h_dim = 400
z_dim = 20
num_epochs = 15
learning_rate = 1e-3

model = VAE(h_dim=h_dim, z_dim=z_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Here for the loss, instead of MSE for the reconstruction loss, we take Binary Cross-Entropy (you should sum the loss over all pixels and samples, not average!)

The KL divergence can be computed as:

$$-\frac{1}{2} \sum_{j=1}^J\left(1+\log(\sigma_j^2) - \mu_j^2 -\sigma_j^2\right)$$

see Appendix B in VAE paper or (Doersch, 2016, https://arxiv.org/abs/1606.05908)

In [ ]:
verbose = False
data_dict = { "epoch": [], "total loss": [], "reconstruction": [], "kl_div": [] }

for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # Forward pass
        x = x.to(device).view(-1, IMAGE_SIZE)
        x_reconst, mu, log_var, z = model(x)

        # Compute reconstruction loss and kl divergence
        reconst_loss = F.binary_cross_entropy(x_reconst, x, reduction='sum')
        kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        loss = reconst_loss + kl_div

        # Backprop and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Bookkeeping
        data_dict["total loss"].append(loss.item() / len(x))
        data_dict["reconstruction"].append(reconst_loss.item() / len(x))
        data_dict["kl_div"].append(kl_div.item() / len(x))
        data_dict["epoch"].append(epoch + float(1+i) / len(data_loader))


        if (i+1) % 100 == 0:
            if verbose:
                print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}"
                      .format(epoch+1, num_epochs, i+1, len(data_loader),
                              reconst_loss.item()/len(x), kl_div.item()/len(x)))
            else:
                live_plot(data_dict, x_key="epoch", title="VAE")

live_plot(data_dict, x_key="epoch", title="VAE")

## A.3 - Evaluating results

Let us see how our network reconstructs our last batch. We display pairs of original digits and reconstructed version side by side.

Observe how most reconstructed digits are essentially identical to the original version.
This means that the identity mapping has been learned well.
You should still see some blurry digits very different from the original (resample a couple times if needed).

In [ ]:
plot_reconstruction(model)

Let's see now how our network generates new samples.

In [ ]:
plot_generation(model)

Not great, but we did not train our network for long... That being said, we have no control of the generated digits. In the rest of this notebook, we explore ways to generates zeroes, ones, twos and so on.


As a by-product, we show how our VAE will allow us to do clustering thanks to the Gumbel VAE described below. But before that, we start by cheating a little bit...

# Part B - Cheating with the "conditional" VAE

We would like to generate samples of a given number. We have so far a model for images $X \in [0,1]^{P}$ with $P = 784$ pixels, generated as $X = f_\theta(Z)$ where $Z \sim \mathcal{N}(0,I)$ is a latent with normal distribution. In other to sample a specific digit, we will use a model
$g_\theta : \mathbb{R}^z \times \{0,\ldots,9\} \to [0,1]^P$ and sample images according to $X = g_\theta(Z, Y)$
where $Y \in \{0, \ldots, 9\}$ is a class label.

In the context of variational autoencoding, this is considered cheating,
because it uses external information (the class label) instead of learning the modes from the data.

To build the function $g$, we will simply concatenate the latent representation with
a one-hot encoding of the class. This way, we can use the above architecture with very little modification.

First, write a function transforming a label into its onehot encoding. This function will be used in the training loop (not in the architecture of the neural network!).

In [ ]:
def l_2_onehot(labels, nb_digits=N_CLASSES):
    # take labels (from the dataloader) and return labels onehot-encoded
    ### your code here ###
    batch_size = labels.size(0)
    onehot = torch.zeros(batch_size, nb_digits).to(device)
    #onehot.scatter_(1, labels.view(-1, 1), 1)
    onehot.scatter_(1, labels.unsqueeze(1), 1)
    return onehot

You can test it on a batch.

In [ ]:
(x,labels) = next(iter(data_loader))

In [ ]:
labels

In [ ]:
assert l_2_onehot(labels).shape == (*labels.shape, N_CLASSES)
l_2_onehot(labels)

Now modify the architecture of the VAE where the decoder takes as input the random code concatenated with the onehot encoding of the label, you can use `torch.cat`.

In [ ]:
class VAE_Cond(nn.Module):
    def __init__(self, h_dim, z_dim):
        super(VAE_Cond, self).__init__()
        ### your code here ###
        self.fc1 = nn.Linear(IMAGE_SIZE, h_dim)
        self.fc_mu = nn.Linear(h_dim, z_dim)
        self.fc_log_var = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim + N_CLASSES, h_dim)
        self.fc4 = nn.Linear(h_dim, IMAGE_SIZE)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        mu = self.fc_mu(h)
        log_var = self.fc_log_var(h)
        return mu, log_var

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, l_onehot):
        ### your code here ###
        z_cond = torch.cat([z, l_onehot], dim=1)
        h = F.relu(self.fc3(z_cond))
        return torch.sigmoid(self.fc4(h))

    def forward(self, x, l_onehot):
        ### your code here ###
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconstructed = self.decode(z, l_onehot)
        return x_reconstructed, mu, log_var

Test your new model on a batch:

In [ ]:
model_C = VAE_Cond(h_dim=h_dim, z_dim=z_dim).to(device)
x = x.to(device).view(-1, IMAGE_SIZE)
l_onehot = l_2_onehot(labels).to(device)
x_reconst, mu, log_var = model_C(x, l_onehot)
assert x_reconst.shape == x.shape
assert mu.shape == log_var.shape and mu.shape == (x.shape[0], z_dim)
x_reconst.shape, mu.shape, log_var.shape

Now you can modify the training loop of your network.

Also, scale the KL term in your loss with $\beta$ to strengthen the regularization as explained in the [$\beta$-VAE paper](https://openreview.net/forum?id=Sy2fzU9gl) see formula (4) in the paper.

In [ ]:
def train_C(model, data_loader=data_loader,num_epochs=num_epochs, beta=10., verbose=True):
    nmi_scores = []
    model.train(True)
    data_dict = { "epoch": [], "total loss": [], "reconstruction": [], "kl_div": [] }
    title = "Conditional VAE"

    for epoch in range(num_epochs):
        for i, (x, labels) in enumerate(data_loader):
            x = x.to(device).view(-1, IMAGE_SIZE)
            l_onehot = l_2_onehot(labels).to(device)

            ### your forward code here ###
            x_reconst, mu, log_var = model(x, l_onehot)

            reconst_loss = F.binary_cross_entropy(x_reconst, x, reduction='sum')
            kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            loss = reconst_loss + beta * kl_div

            # Backprop and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Bookkeeping
            data_dict["total loss"].append(loss.item() / len(x))
            data_dict["reconstruction"].append(reconst_loss.item() / len(x))
            data_dict["kl_div"].append(kl_div.item() / len(x))
            data_dict["epoch"].append(epoch + float(1+i) / len(data_loader))


            if (i+1) % 100 == 0:
                if verbose:
                    print("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}"
                           .format(epoch+1, num_epochs, i+1, len(data_loader),
                                   reconst_loss.item()/len(x),
                                   kl_div.item()/len(x)))
                else:
                    live_plot(data_dict, x_key="epoch", title=title)

    live_plot(data_dict, x_key="epoch", title=title)

In [ ]:
model_C = VAE_Cond(h_dim=h_dim, z_dim=z_dim).to(device)
optimizer = torch.optim.Adam(model_C.parameters(), lr=learning_rate)

In [ ]:
train_C(model_C, num_epochs=15, verbose=False)

In [ ]:
plot_conditional_generation(model_C)

Here you should get nice results. Now we will avoid the use of the labels...

# Part C - No cheating with Gumbel VAE

Implement a VAE where you add a categorical variable $c\in \{0,\dots 9\}$ so that your latent variable model is $p(c,z,x) = p(c)p(z)p_{\theta}(x|c,z)$ and your variational posterior is $q_{\phi}(c|x)q_{\phi}(z|x)$ as described in this NeurIPS paper: [(Dupont, 2018)](https://arxiv.org/abs/1804.00104). Try to make only minimal modifications to previous architecture.


The idea is to incorporate a categorical variable in your latent space, without cheating by forcing it to take the value of the image's label. You hope that this categorical variable will encode the class of the digit, so that your network can use it for a better reconstruction, but can't force it. Moreover, if things work as planned, you will then be able to generate digits conditionally to the class, i.e. you can choose the class thanks to the latent categorical variable $c$ and then generate digits from this class.

## C.1 - Gumbel trick for discrete latent variables

As noticed above, in order to sample random variables while still being able to use backpropagation, we need to use the reparameterization trick which is easy for Gaussian random variables. For categorical random variables, the reparameterization trick is explained in [(Jang et al., 2016)](https://arxiv.org/abs/1611.01144). This is implemented in PyTorch thanks to [F.gumbel_softmax](https://pytorch.org/docs/stable/generated/torch.nn.functional.gumbel_softmax.html).

Note: there is an instability in the PyTorch `F.gumbel_softmax` implementation. We provide instead `gumbel_softmax` which takes the same arguments and ensures that the output is not NaN. If you encounter nans in your training or errors in the binary cross-entropy, make sure you are using the stabilized version.

Note: `gumbel_softmax` expects log-probabilities as input. See https://pytorch.org/docs/stable/generated/torch.nn.LogSoftmax.html

In [ ]:
def gumbel_softmax(logits, tau=1, hard=False, eps=1e-10, dim=-1):
    # type: (Tensor, float, bool, float, int) -> Tensor
    r"""
    This is a stabilized version of PyTorch's `F.gumbel_softmax`
    function, which samples zeros from the exponential and the
    nans when taking the logarithm.
    """

    def stabilized_gumbel():
        gumbels = - torch.empty_like(logits).exponential_().log()
        reject = torch.isnan(gumbels).sum() or torch.isinf(gumbels).sum()
        return stabilized_gumbel() if reject else gumbels

    gumbels = stabilized_gumbel()  # ~Gumbel(0,1)
    gumbels = (logits + gumbels) / tau  # ~Gumbel(logits,tau)
    y_soft = gumbels.softmax(dim)

    if hard:
        # Straight through.
        index = y_soft.max(dim, keepdim=True)[1]
        y_hard = torch.zeros_like(logits).scatter_(dim, index, 1.0)
        ret = y_hard - y_soft.detach() + y_soft
    else:
        # Reparametrization trick.
        ret = y_soft
    return ret

In [ ]:
class VAE_Gumbel(nn.Module):
    def __init__(self, h_dim, z_dim):
        super(VAE_Gumbel, self).__init__()
        ### your code here ###
        self.fc1 = nn.Linear(IMAGE_SIZE, h_dim)
        self.fc_mu = nn.Linear(h_dim, z_dim)
        self.fc_log_var = nn.Linear(h_dim, z_dim)
        self.fc_c = nn.Linear(h_dim, N_CLASSES)

        self.fc3 = nn.Linear(z_dim + N_CLASSES, h_dim)
        self.fc4 = nn.Linear(h_dim, IMAGE_SIZE)

    def encode(self, x):
        ### your code here ###
        h = F.relu(self.fc1(x))
        mu = self.fc_mu(h)
        log_var = self.fc_log_var(h)
        logits_c = self.fc_c(h)
        return mu, log_var, logits_c

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, y_onehot):
        ### your code here ###
        z_cond = torch.cat([z, y_onehot], dim=1)
        h = F.relu(self.fc3(z_cond))
        return torch.sigmoid(self.fc4(h))

    def forward(self, x):
        ### your code here ###
        mu, log_var, logits_c = self.encode(x)
        z = self.reparameterize(mu, log_var)
        c = gumbel_softmax(logits_c)
        x_reconstructed = self.decode(z, c)
        return x_reconstructed, mu, log_var, c, logits_c

In [ ]:
model_G = VAE_Gumbel(h_dim=h_dim, z_dim=z_dim).to(device)
optimizer = torch.optim.Adam(model_G.parameters(), lr=learning_rate)

You need to modify the loss to take into account the categorical random variable with an uniform prior on $\{0,\dots 9\}$, see Appendix A.2 in [(Dupont, 2018)](https://arxiv.org/abs/1804.00104).

Note this can be implemented by taking the entropy of $c$

In [ ]:
def train_G(model, data_loader=data_loader,num_epochs=num_epochs, beta = 1., verbose=True):
    nmi_scores = []
    model.train(True)
    data_dict = { "epoch": [], "total loss": [], "reconstruction": [], "kl_div": [], "entropy": [] }
    title = "Gumbel VAE"

    for epoch in range(num_epochs):
        all_labels = []
        all_labels_est = []
        for i, (x, labels) in enumerate(data_loader):
            x = x.to(device).view(-1, IMAGE_SIZE)
            ### your forward code here ###
            x_reconst, mu, log_var, c, logits_c = model(x)

            reconst_loss = F.binary_cross_entropy(x_reconst, x, reduction='sum')
            kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            entropy = -torch.sum(c * torch.log(c + 1e-10))
            loss = reconst_loss + beta * kl_div - entropy

            # Backprop and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Bookkeeping
            data_dict["total loss"].append(loss.item() / len(x))
            ### your bookkeeping code here ###
            data_dict["epoch"].append(epoch + float(1+i) / len(data_loader))
            data_dict["reconstruction"].append(reconst_loss.item() / len(x))
            data_dict["kl_div"].append(kl_div.item() / len(x))

            if (i+1) % 100 == 0:
                if verbose:
                    print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}, Entropy: {:.4f}"
                           .format(epoch+1, num_epochs, i+1, len(data_loader),
                                   reconst_loss.item()/len(x),
                                   kl_div.item()/len(x),
                                   entropy.item()/len(x)))
                else:
                    live_plot(data_dict, x_key="epoch", title=title)

    live_plot(data_dict, x_key="epoch", title=title)

In [ ]:
train_G(model_G, num_epochs=20, verbose=False)

In [ ]:
plot_reconstruction(model_G)

The reconstruction is good, but we care more about the generation. For each category, we generate 8 samples thanks to the `plot_conditional_generation()` function.
Consistently with the previous use, each row is supposed to consist of the same digit sampled 8 times, and each row should correspond to a distinct digit.

In [ ]:
plot_conditional_generation(model_G)

It does not look like our original idea is working...

What is happening is that our network is not using the categorical variable at all (all "digits" look the same, the variation comes from the re-sampling of Z). We can track the [normalized mutual information](https://en.wikipedia.org/wiki/Mutual_information#Normalized_variants) (see [this method in scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html)) between the true labels and the labels predicted by our network (just by taking the category with maximal probability).

Change your training loop to return the normalized mutual information (NMI) for each epoch. Plot the curve to check that the NMI is actually decreasing.

In [ ]:
def train_G_NMI(model, data_loader=data_loader,num_epochs=num_epochs, beta = 1., verbose=True):
    nmi_scores = []
    model.train(True)
    data_dict = { "epoch": [], "total loss": [], "reconstruction": [], "kl_div": [], "entropy": [] }
    title = "Gumbel VAE NMI"

    for epoch in range(num_epochs):
        all_labels = []
        all_labels_est = []
        for i, (x, labels) in enumerate(data_loader):
            x = x.to(device).view(-1, IMAGE_SIZE)
            ### your forward code here ###
            x_reconst, mu, log_var, c, logits_c = model(x)

            reconst_loss = F.binary_cross_entropy(x_reconst, x, reduction='sum')
            kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            probs = F.softmax(logits_c, dim=1)
            entropy = -torch.sum(probs * torch.log(probs + 1e-10))
            loss = reconst_loss + beta * kl_div - entropy

            # Backprop and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Bookkeeping
            data_dict["total loss"].append(loss.item() / len(x))
            ### your bookkeeping code here ###
            data_dict["reconstruction"].append(reconst_loss.item() / len(x))
            data_dict["kl_div"].append(kl_div.item() / len(x))
            data_dict["entropy"].append(entropy.item() / len(x))
            data_dict["epoch"].append(epoch + float(1+i) / len(data_loader))
            #all_labels.append(labels.numpy())
            #_, predicted_labels = torch.max(c, 1)
            #all_labels_est.append(predicted_labels.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            all_labels_est.append(probs.argmax(dim=1).cpu().numpy())

            if (i+1) % 100 == 0:
                if verbose:
                    print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}, Entropy: {:.4f}"
                           .format(epoch+1, num_epochs, i+1, len(data_loader),
                                   reconst_loss.item()/len(x),
                                   kl_div.item()/len(x),
                                   entropy.item()/len(x)))
                else:
                    live_plot(data_dict, x_key="epoch", title=title)
        true_labels = np.concatenate(all_labels)
        pred_labels = np.concatenate(all_labels_est)
        nmi = normalized_mutual_info_score(true_labels, pred_labels)
        nmi_scores.append(nmi)
        #data_dict["nmi"].append(nmi)
        print(f"Epoch {epoch+1}, NMI: {nmi:.4f}")

    live_plot(data_dict, x_key="epoch", title=title)
    return nmi_scores

In [ ]:
model_G = VAE_Gumbel(h_dim=h_dim, z_dim=z_dim).to(device)
optimizer = torch.optim.Adam(model_G.parameters(), lr=learning_rate)

In [ ]:
nmi = train_G_NMI(model_G, num_epochs=20, verbose=False)

plt.plot(nmi)
plt.xlabel("Epoch")
plt.ylabel("NMI")
plt.title("NMI Curve")
plt.grid(alpha=0.5, which='both')
plt.show()

## C.2 - Robust disentangling with controlled capacity increase

This problem is explained in [(Burgess et al., 2018)](https://arxiv.org/abs/1804.03599) and a solution is proposed in Section 5.

In order to force our network to use the categorical variable, we will change the loss according to [(Dupont, 2018)](https://arxiv.org/abs/1804.00104), Section 3 Equation (7).

Implement this change in the training loop and plot the new NMI curve. Increase the $C_z$ and $C_c$ constants by a constant value every epoch until it reaches the value passed in argument.
For $\beta = 20, C_z=100, C_c=100$, you should see that NMI increases.

In [ ]:
model_G = VAE_Gumbel(h_dim=h_dim, z_dim=z_dim).to(device)
optimizer = torch.optim.Adam(model_G.parameters(), lr=learning_rate)

In [ ]:
def train_G_modified_loss(model, data_loader=data_loader,num_epochs=num_epochs, beta=1. , C_z_fin=0, C_c_fin=0, verbose=True):
    ### your code here ###
    nmi_scores = []
    model.train(True)
    data_dict = {"epoch": [], "total loss": [], "reconstruction": [], "kl_div": [], "entropy": []}
    title = "Gumbel VAE with Modified Loss"
    #C_z_step = C_z_fin / num_epochs if num_epochs > 0 else 0
    #C_c_step = C_c_fin / num_epochs if num_epochs > 0 else 0

    C_z = 0
    C_c = 0

    for epoch in range(num_epochs):
        all_labels = []
        all_labels_est = []

        for i, (x, labels) in enumerate(data_loader):
            x = x.to(device).view(-1, IMAGE_SIZE)
            labels = labels.to(device)

            x_reconst, mu, log_var, c, logits_c = model(x)
            reconst_loss = F.binary_cross_entropy(x_reconst, x, reduction='sum')
            #kl_div_z = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            #kl_div_z_loss = beta * torch.abs(kl_div_z - C_z)
            kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            probs = F.softmax(logits_c, dim=1)
            entropy = -torch.sum(probs * torch.log(probs + 1e-10))
            entropy = -torch.sum(c * torch.log(c + 1e-10))
            C_z = min(C_z + C_z_fin / num_epochs, C_z_fin)
            C_c = min(C_c + C_c_fin / num_epochs, C_c_fin)
            loss = reconst_loss + beta  *(kl_div - C_z) + (entropy - C_c)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            all_labels.append(labels.cpu().numpy())
            all_labels_est.append(probs.argmax(dim=1).cpu().numpy())

            # Bookkeeping
            data_dict["total loss"].append(loss.item() / len(x))
            data_dict["reconstruction"].append(reconst_loss.item() / len(x))
            data_dict["kl_div"].append(kl_div.item() / len(x))
            data_dict["entropy"].append(entropy.item() / len(x))
            data_dict["epoch"].append(epoch + float(1+i) / len(data_loader))

            #all_labels.append(labels.cpu().numpy())
            #pred_labels = c.argmax(dim=1).cpu().numpy()
            #all_labels_est.append(pred_labels)

            if (i+1) % 100 == 0:
                if verbose:
                    print(f"Epoch[{epoch+1}/{num_epochs}], Step [{i+1}/{len(data_loader)}], "
                          f"C_z: {C_z:.1f}, C_c: {C_c:.1f}, "
                          f"Reconst Loss: {reconst_loss.item()/len(x):.4f}, "
                          f"KL Z: {kl_div.item()/len(x):.4f}, "
                          f"Entropy: {entropy.item()/len(x):.4f}")
                else:
                    live_plot(data_dict, x_key="epoch", title=title)

        true_labels = np.concatenate(all_labels)
        pred_labels = np.concatenate(all_labels_est)
        nmi = normalized_mutual_info_score(true_labels, pred_labels)
        nmi_scores.append(nmi)
        print(f"Epoch {epoch+1}, C_z: {C_z:.1f}, C_c: {C_c:.1f}, NMI: {nmi:.4f}")

    live_plot(data_dict, x_key="epoch", title=title)
    return nmi_scores

In [ ]:
# Hyper-parameters
num_epochs = 40
learning_rate = 1e-3
beta = 20
C_z_fin = 100
C_c_fin = 100

model_G = VAE_Gumbel(h_dim=h_dim, z_dim = z_dim).to(device)
optimizer = torch.optim.Adam(model_G.parameters(), lr=learning_rate)

nmi = train_G_modified_loss(
    model_G, data_loader,
    num_epochs=num_epochs,
    beta=beta,
    C_z_fin=C_z_fin,
    C_c_fin=C_c_fin,
    verbose=False)

In [ ]:
plt.plot(nmi)
plt.xlabel("epoch")
plt.ylabel("Normalized Mutual Information")
plt.grid(alpha=.5, which='both')

In [ ]:
plot_reconstruction(model_G)

In [ ]:
plot_conditional_generation(model_G, fix_number=None)

In [ ]:
plot_conditional_generation(model_G, fix_number=2)

In [ ]:
plot_conditional_generation(model_G, fix_number=8)

## C.3 - Interpretation of learned results

Compare the generated digits from the Conditional VAE model and the modified Gumbel-VAE.
The conditional VAE produces all digits in order, which makes for a nice picture; on the other hand, the Gumbel VAE has no way of knowing the order of digits, it must just guess modes of pictures that look the same, so the order is meaningless.
Note however the variations of straight/inclined ones, are they present on the same row ?
What about shapes from the same row morphing from one digit to another, does it make sense that this was a considered a single mode by the model ? Are there distinct modes corresponding to the same digit, and if so, is it a reasonable distinction ?

In [ ]:
plot_conditional_generation(model_C, fix_number=None)

In [ ]:
plot_conditional_generation(model_G, fix_number=None)

---

Le VAE conditionnel produit des chiffres ordonnées car il utilise les étiquettes des classes pendant l'entraînement. Pour Gumbel VAE modifié, il doit découvrir lui même les caractéristiques car il n'a pas accès aux labels. Le premier est supervisé et le second non supervisé.

Dans une même ligne, on observe que les résultats du Gumbel VAE modifié, présentent les variations du même chiffre sur la même ligne. On voit des chiffres avec une rotation différentes regroupés ensemble (0 et 1 par exemple). Ce qui montre que les chiffres malgré une inclinaison différente sont reconnus et que le modèle a appris les différents caractéristiques des chiffres.

On observe aussi que certains chiffres sont floutés ou que les formes se transforment et peuvent ressemblés à d'autres chiffres notamment le 3, 5 et 8. Cela est dut au modèle qui découvre des structures similaires entre ces classes; c'est pour cette raison qu'ils peuvent être sur la même ligne.

Il y a des modes distincts qui peuvent être en réalité le même chiffre et c'est explicable lorsque les chiffres sont écrits avec des angles très différents. Ou bien le 7 qui peut ressembler au 1 en fonction de son écriture. Il est donc raisonnable que le Gumbel VAE modifié assigne différents modes au même chiffre.